# Lab 05: Threat Intelligence Agent

Build an AI agent that gathers and correlates threat intelligence from multiple sources.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/depalmar/ai_for_the_win/blob/main/notebooks/lab05_threat_intel_agent.ipynb)

## Learning Objectives
- Building AI agents with tools
- Threat intelligence data sources
- IOC correlation and enrichment
- MITRE ATT&CK mapping

In [ ]:
# Install dependencies
# !pip install anthropic

In [ ]:
import json
from typing import List, Dict, Any
from dataclasses import dataclass
from enum import Enum

## 1. Define Tool Schemas

In [ ]:
# Tool definitions for the agent
TOOLS = [
    {
        "name": "lookup_ip",
        "description": "Look up threat intelligence for an IP address",
        "input_schema": {
            "type": "object",
            "properties": {
                "ip": {"type": "string", "description": "IP address to look up"}
            },
            "required": ["ip"]
        }
    },
    {
        "name": "lookup_hash",
        "description": "Look up threat intelligence for a file hash",
        "input_schema": {
            "type": "object",
            "properties": {
                "hash": {"type": "string", "description": "File hash (MD5, SHA1, SHA256)"}
            },
            "required": ["hash"]
        }
    },
    {
        "name": "lookup_domain",
        "description": "Look up threat intelligence for a domain",
        "input_schema": {
            "type": "object",
            "properties": {
                "domain": {"type": "string", "description": "Domain name to look up"}
            },
            "required": ["domain"]
        }
    },
    {
        "name": "get_mitre_technique",
        "description": "Get details about a MITRE ATT&CK technique",
        "input_schema": {
            "type": "object",
            "properties": {
                "technique_id": {"type": "string", "description": "MITRE technique ID (e.g., T1566)"}
            },
            "required": ["technique_id"]
        }
    }
]

## 2. Mock Threat Intel Database

In [ ]:
# Mock threat intelligence data
THREAT_INTEL_DB = {
    "ips": {
        "185.143.223.47": {
            "classification": "Malicious",
            "confidence": 95,
            "threat_types": ["C2", "Malware Distribution"],
            "malware_family": "Cobalt Strike",
            "country": "Russia",
            "first_seen": "2023-06-15",
            "last_seen": "2024-01-15",
            "related_campaigns": ["APT29", "Cozy Bear"]
        },
        "203.0.113.50": {
            "classification": "Suspicious",
            "confidence": 70,
            "threat_types": ["Scanning"],
            "country": "China",
            "first_seen": "2024-01-10"
        }
    },
    "hashes": {
        "d41d8cd98f00b204e9800998ecf8427e": {
            "classification": "Malicious",
            "malware_family": "Emotet",
            "file_type": "PE32",
            "first_seen": "2023-11-01",
            "detection_ratio": "58/72",
            "mitre_techniques": ["T1566.001", "T1059.001", "T1547.001"]
        }
    },
    "domains": {
        "malware-c2.evil.com": {
            "classification": "Malicious",
            "threat_types": ["C2", "Phishing"],
            "associated_ips": ["185.143.223.47"],
            "registrar": "Anonymous Registrar",
            "created": "2023-12-01"
        }
    },
    "mitre": {
        "T1566.001": {
            "name": "Spearphishing Attachment",
            "tactic": "Initial Access",
            "description": "Adversaries may send spearphishing emails with a malicious attachment",
            "detection": "Monitor for suspicious email attachments, Office macros",
            "mitigations": ["User Training", "Email Filtering", "Disable Macros"]
        },
        "T1059.001": {
            "name": "PowerShell",
            "tactic": "Execution",
            "description": "Adversaries may abuse PowerShell for execution",
            "detection": "Monitor PowerShell logs, encoded commands",
            "mitigations": ["Script Block Logging", "Constrained Language Mode"]
        }
    }
}

## 3. Tool Implementation

In [ ]:
def lookup_ip(ip: str) -> Dict:
    """Look up IP in threat intel database."""
    if ip in THREAT_INTEL_DB["ips"]:
        return THREAT_INTEL_DB["ips"][ip]
    return {"classification": "Unknown", "message": "No threat data found"}

def lookup_hash(hash_value: str) -> Dict:
    """Look up file hash in threat intel database."""
    if hash_value in THREAT_INTEL_DB["hashes"]:
        return THREAT_INTEL_DB["hashes"][hash_value]
    return {"classification": "Unknown", "message": "No threat data found"}

def lookup_domain(domain: str) -> Dict:
    """Look up domain in threat intel database."""
    if domain in THREAT_INTEL_DB["domains"]:
        return THREAT_INTEL_DB["domains"][domain]
    return {"classification": "Unknown", "message": "No threat data found"}

def get_mitre_technique(technique_id: str) -> Dict:
    """Get MITRE ATT&CK technique details."""
    if technique_id in THREAT_INTEL_DB["mitre"]:
        return THREAT_INTEL_DB["mitre"][technique_id]
    return {"error": f"Technique {technique_id} not found"}

def execute_tool(name: str, args: Dict) -> str:
    """Execute a tool and return result."""
    if name == "lookup_ip":
        result = lookup_ip(args["ip"])
    elif name == "lookup_hash":
        result = lookup_hash(args["hash"])
    elif name == "lookup_domain":
        result = lookup_domain(args["domain"])
    elif name == "get_mitre_technique":
        result = get_mitre_technique(args["technique_id"])
    else:
        result = {"error": f"Unknown tool: {name}"}
    
    return json.dumps(result, indent=2)

## 4. Test Tools

In [ ]:
# Test the tools
print("IP Lookup:")
print(execute_tool("lookup_ip", {"ip": "185.143.223.47"}))

print("\nDomain Lookup:")
print(execute_tool("lookup_domain", {"domain": "malware-c2.evil.com"}))

print("\nMITRE Technique:")
print(execute_tool("get_mitre_technique", {"technique_id": "T1566.001"}))

## 5. Agent Loop

In [ ]:
class ThreatIntelAgent:
    """AI agent for threat intelligence gathering."""
    
    def __init__(self):
        try:
            from anthropic import Anthropic
            self.client = Anthropic()
            self.available = True
        except:
            self.available = False
    
    def investigate(self, query: str) -> str:
        """Investigate a threat using available tools."""
        if not self.available:
            return self._mock_investigation(query)
        
        messages = [{"role": "user", "content": query}]
        
        # Agent loop
        while True:
            response = self.client.messages.create(
                model="claude-sonnet-4-20250514",
                max_tokens=1024,
                tools=TOOLS,
                messages=messages
            )
            
            # Check if we need to call tools
            if response.stop_reason == "tool_use":
                # Execute tools
                tool_results = []
                for content in response.content:
                    if content.type == "tool_use":
                        result = execute_tool(content.name, content.input)
                        tool_results.append({
                            "type": "tool_result",
                            "tool_use_id": content.id,
                            "content": result
                        })
                
                # Add to messages
                messages.append({"role": "assistant", "content": response.content})
                messages.append({"role": "user", "content": tool_results})
            else:
                # Final response
                return response.content[0].text
    
    def _mock_investigation(self, query: str) -> str:
        return """
## Threat Intelligence Report

### Investigation Summary
Based on the IOCs provided, I've gathered the following intelligence:

### IP Analysis: 185.143.223.47
- **Classification**: Malicious (95% confidence)
- **Threat Types**: C2 Server, Malware Distribution
- **Malware Family**: Cobalt Strike
- **Associated Campaigns**: APT29, Cozy Bear
- **Location**: Russia

### Related MITRE Techniques
- T1566.001: Spearphishing Attachment (Initial Access)
- T1059.001: PowerShell (Execution)

### Recommendations
1. Block IP at perimeter firewall
2. Search for this IP in historical logs
3. Enable PowerShell script block logging
4. Alert on Cobalt Strike beacon patterns
"""

In [ ]:
# Run investigation
agent = ThreatIntelAgent()
result = agent.investigate(
    "Investigate IP 185.143.223.47 and provide threat intelligence with MITRE mappings"
)
print(result)

## Summary

In this lab, we built a threat intelligence agent with:

1. **Tool Definitions** - Structured schemas for API-like tools
2. **Mock Database** - Simulated threat intel data
3. **Agent Loop** - Autonomous tool execution
4. **MITRE Mapping** - Technique correlation

### Key Takeaways:
- Agents can chain multiple tool calls
- Tool definitions guide LLM behavior
- MITRE ATT&CK provides common language
- Correlation across sources adds value

### Next Steps:
1. Add real threat intel APIs (VirusTotal, Shodan)
2. Implement caching for repeated lookups
3. Build automated IOC enrichment pipeline